In [1]:
%load_ext blackcellmagic
%alias_magic -c -p "-l 120" b black

Created `%%b` as an alias for `%%black -l 120`.


# MEI Demo

In [2]:
import datajoint as dj

dj.config["enable_python_native_blobs"] = True
dj.config["schema_name"] = "nnfabrik_tutorial"

schema = dj.schema("nnfabrik_tutorial")

Connecting christoph@sinzlab.chlkmukhxp6i.eu-central-1.rds.amazonaws.com:3306


In [3]:
import os

from matplotlib import pyplot as plt
from torch import load

from featurevis.main import TrainedEnsembleModelTemplate, CSRFV1SelectorTemplate, MEISeed, MEIMethod, MEITemplate
from nnfabrik.template import TrainedModelBase
from nnfabrik.main import Dataset

## 1. Define Tables

In [4]:
@schema
class TrainedModel(TrainedModelBase):
    pass


@schema
class TrainedEnsembleModel(TrainedEnsembleModelTemplate):
    dataset_table = Dataset
    trained_model_table = TrainedModel


@schema
class CSRFV1Selector(CSRFV1SelectorTemplate):
    dataset_table = Dataset


@schema
class MEI(MEITemplate):
    trained_model_table = TrainedEnsembleModel
    selector_table = CSRFV1Selector

## 2. Reset Tables For Demo

In [5]:
CSRFV1Selector().drop()
TrainedEnsembleModel().drop()
MEIMethod().drop()
MEISeed().drop()

`nnfabrik_tutorial`.`__c_s_r_f_v1_selector` (14 tuples)
`nnfabrik_tutorial`.`__m_e_i` (1 tuples)


Proceed? [yes, No]:  yes


Tables dropped.  Restart kernel.
`nnfabrik_tutorial`.`trained_ensemble_model` (1 tuples)
`nnfabrik_tutorial`.`trained_ensemble_model__member` (2 tuples)


Proceed? [yes, No]:  yes


Tables dropped.  Restart kernel.
`nnfabrik_tutorial`.`#m_e_i_method` (1 tuples)


Proceed? [yes, No]:  yes


Tables dropped.  Restart kernel.
`nnfabrik_tutorial`.`#m_e_i_seed` (1 tuples)


Proceed? [yes, No]:  yes


Tables dropped.  Restart kernel.


## 3. Create Ensemble Model

In [5]:
TrainedEnsembleModel().create_ensemble(dict(dataset_fn="csrf_v1"), comment="Happy little ensemble")
TrainedEnsembleModel()

dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,ensemble_hash the hash of the ensemble,ensemble_comment a short comment describing the ensemble
csrf_v1,3d94500a46b792bbb480aedfc30f9753,bd42401406ec0db224849255505fa17d,Happy little ensemble


## 4. Populate Selector Table

In [6]:
CSRFV1Selector.populate()
CSRFV1Selector()

dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,neuron_id unique neuron identifier,neuron_position integer position of the neuron in the model's output,session_id unique session identifier
csrf_v1,3d94500a46b792bbb480aedfc30f9753,188,0,3638456653849
csrf_v1,3d94500a46b792bbb480aedfc30f9753,189,1,3638456653849
csrf_v1,3d94500a46b792bbb480aedfc30f9753,371,0,3634744023164
csrf_v1,3d94500a46b792bbb480aedfc30f9753,372,1,3634744023164
csrf_v1,3d94500a46b792bbb480aedfc30f9753,373,2,3634744023164
csrf_v1,3d94500a46b792bbb480aedfc30f9753,374,3,3634744023164
csrf_v1,3d94500a46b792bbb480aedfc30f9753,375,4,3634744023164
csrf_v1,3d94500a46b792bbb480aedfc30f9753,376,5,3634744023164
csrf_v1,3d94500a46b792bbb480aedfc30f9753,377,6,3634744023164
csrf_v1,3d94500a46b792bbb480aedfc30f9753,378,7,3634744023164


## 5. Specify MEI Method Parameters

In [ ]:
MEI

In [7]:
method_fn = "featurevis.methods.ascend_gradient"
method_config = dict(
    optimizer="torch.optim.SGD",
    optimizer_kwargs=dict(lr=0.1),
    stopper="featurevis.stoppers.NumIterations",
    stopper_kwargs=dict(num_iterations=1000),
    transform=None,
    transform_kwargs=None,
    regularization=None,
    regularization_kwargs=None,
    precondition=None,
    precondition_kwargs=None,
    postprocessing=None,
    postprocessing_kwargs=None,
    device="cuda",
)
MEIMethod().add_method(method_fn, method_config)
MEIMethod()

method_fn name of the method function,method_hash hash of the method config,method_config method configuration object,method_ts UTZ timestamp at time of insertion
featurevis.methods.ascend_gradient,acfff7083e0aa299f6101c7a3f18efc3,=BLOB=,2020-04-21 09:49:54


## 6. Add Seed

In [8]:
MEISeed().insert1(dict(mei_seed=42))
MEISeed()

mei_seed MEI seed
42


## 7. Generate MEIs

In [ ]:
MEI().populate(dict(neuron_id=188), display_progress=True)
MEI()

## 8. Look at MEIs

In [ ]:
neuron_id = 188
mei_path = (MEI() & dict(ensemble_id=0, neuron_id=neuron_id)).fetch1("mei")
plt.imshow(load(mei_path), cmap="gray")
plt.gca().axis("off")
os.remove(mei_path)